<a href="https://colab.research.google.com/github/muskaanfayyaz/Tracing/blob/main/basic_tracing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uq openai-agents
!pip install -Uq openai openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.2/167.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.2/160.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.8/786.8 kB 16.6 MB/s eta 0:00:00


In [3]:
import nest_asyncio
nest_asyncio.apply()
from google.colab import userdata

GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    raise ValueError("❌ No GEMINI_API_KEY found in Colab userdata. Go to Colab → Tools → Variables and set it.")


In [5]:
from agents import Agent, Runner, trace, OpenAIChatCompletionsModel, RunConfig
from openai import AsyncOpenAI
import asyncio

from agents import Agent, Runner, trace

# Create OpenAI-compatible Gemini client
external_client = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# Define model using Gemini
model = OpenAIChatCompletionsModel(
    model="gemini-2.5-flash",
    openai_client=external_client
)

# Run configuration
config = RunConfig(
    model=model,
    tracing_disabled=True
)


async def main():
    agent = Agent(name="Joke generator", instructions="Tell funny jokes.")

    with trace("Joke workflow"):
        first_result = await Runner.run(agent, "Tell me a joke", run_config=config)
        second_result = await Runner.run(agent, f"Rate this joke: {first_result.final_output}", run_config=config)
        print(f"Joke: {first_result.final_output}")
        print(f"Rating: {second_result.final_output}")

asyncio.run(main())

Joke: Why did the scarecrow win an award?

Because he was outstanding in his field!
Rating: That's a classic! It's definitely a **solid 4 out of 5** in the "dad joke" category.

Here's why:

*   **Clever Pun:** The play on "outstanding" (excellent vs. literally standing out) and "in his field" (area of expertise vs. a literal field) is well-executed and perfectly aligned with the scarecrow.
*   **Harmless & Family-Friendly:** It's the kind of joke you can tell anyone, anywhere.
*   **Predictable but Satisfying:** You can often see the punchline coming with these types of jokes, but that's part of their charm. It delivers exactly what you expect from a good, clean pun.

It gets a chuckle and a slight groan (the good kind!). Well done!


In [6]:
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel, function_tool

# 1. Which LLM Service?
external_client: AsyncOpenAI = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# 2. Which LLM Model?
llm_model: OpenAIChatCompletionsModel = OpenAIChatCompletionsModel(
    model="gemini-2.5-flash",
    openai_client=external_client
)

@function_tool
def get_weather(city: str) -> str:
    """A simple function to get the weather for a user."""
    return f"The weather for {city} is sunny."


base_agent: Agent = Agent(
    name="WeatherAgent",
    instructions="You are a helpful assistant.",
    model=llm_model,
    tools=[get_weather]
)

new_agent: Agent = Agent(
    name="WeatherAgent",
    instructions="You are a helpful assistant.",
    model=llm_model,
    tools=[get_weather]
)

res = Runner.run_sync(base_agent, "What's the weather in Karachi?")
print(res)

# Now check the trace in

RunResult:
- Last agent: Agent(name="WeatherAgent", ...)
- Final output (str):
    The weather in Karachi is sunny.
- 3 new item(s)
- 2 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
